In [19]:
from __future__ import print_function
from show3d_balls import showpoints
import argparse
import numpy as np
import torch
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
from pointnet.dataset_custom import CustomDataset
from pointnet.model import PointNetDenseCls
import matplotlib.pyplot as plt
import easydict

In [20]:
opt = easydict.EasyDict({'model': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/seg/seg_model_Head_49.pth',
                         'dataset': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data',
                         'class_choice': 'Head',
                        })

In [21]:
idx = 2

In [22]:
d = CustomDataset(root=opt.dataset, 
                        classification=False, 
                        data_augmentation=False,
                       split='test')

In [23]:
print("model %d/%d" % (idx, len(d)))
point, seg = d[idx]
print(point.size(), seg.size())
point_np = point.numpy()

model 2/7
torch.Size([2500, 3]) torch.Size([2500])


In [24]:
cmap = plt.cm.get_cmap("hsv", 10)
cmap = np.array([cmap(i) for i in range(10)])[:, :3]
gt = cmap[seg.numpy() - 1, :]


In [25]:
state_dict = torch.load(opt.model)
classifier = PointNetDenseCls(k= state_dict['conv4.weight'].size()[0])
classifier.load_state_dict(state_dict,strict=False)
classifier.eval()

PointNetDenseCls(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  

In [26]:
point = point.transpose(1, 0).contiguous()

point = Variable(point.view(1, point.size()[0], point.size()[1]))
pred, _, _ = classifier(point)
pred_choice = pred.data.max(2)[1]
print(pred_choice)

tensor([[0, 0, 0,  ..., 0, 0, 0]])


In [36]:
_list = pred_choice.tolist()[0]

In [39]:
zeros = _list.count(0)

In [40]:
print(zeros)

2500


In [9]:
#print(pred_choice.size())
pred_color = cmap[pred_choice.numpy()[0], :]

#print(pred_color.shape)
showpoints(point_np, gt, pred_color)

113